In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

In [6]:
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,20000.0,17.19%,499.10,RENT,47000.0,Source Verified,Mar-2019,Issued,n,14.02,...,98.0,12.5,0.0,0.0,75824.0,31546.0,33800.0,21524.0,N,N
1,21225.0,14.74%,502.05,MORTGAGE,225000.0,Not Verified,Mar-2019,Issued,n,16.80,...,100.0,50.0,1.0,0.0,747075.0,209426.0,53500.0,128175.0,N,N
2,5000.0,17.97%,180.69,MORTGAGE,62000.0,Not Verified,Mar-2019,Issued,n,19.82,...,66.7,50.0,0.0,0.0,255738.0,31615.0,9400.0,39938.0,N,N
3,20000.0,8.19%,628.49,MORTGAGE,200000.0,Not Verified,Mar-2019,Issued,n,22.66,...,95.7,22.2,0.0,0.0,448069.0,84744.0,49400.0,105180.0,N,N
4,12000.0,15.57%,289.09,MORTGAGE,49000.0,Source Verified,Mar-2019,Issued,n,13.47,...,86.1,14.3,0.0,0.0,189260.0,106025.0,24400.0,68860.0,N,N


In [7]:
df = df.dropna(axis='columns', how='all')

In [8]:
df = df.dropna()

In [9]:
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

In [10]:
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100

In [11]:
x = {'Current': 'low_risk'}   
df = df.replace(x)

In [12]:
x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

In [13]:
df.reset_index(inplace=True, drop=True)

In [14]:
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


Split Data

In [15]:
X = df.drop('loan_status', axis=1)
X = pd.get_dummies(X)

In [16]:
y = df['loan_status']
type(X)

pandas.core.frame.DataFrame

In [17]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [18]:
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [21]:
X_train.shape

(51612, 95)

In [22]:
from imblearn.over_sampling import RandomOverSampler

In [23]:
ros = RandomOverSampler(random_state=1)

In [24]:
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [25]:
from sklearn.linear_model import LogisticRegression

In [26]:
model = LogisticRegression(solver='lbfgs', random_state=1)

In [27]:
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [28]:
from sklearn.metrics import confusion_matrix

In [29]:
y_pred = model.predict(X_test)

In [30]:
confusion_matrix(y_test, y_pred)

array([[   54,    33],
       [ 5945, 11173]], dtype=int64)

In [31]:
from sklearn.metrics import balanced_accuracy_score

In [32]:
balanced_accuracy_score(y_test, y_pred)

0.6366972052004142

In [33]:
from imblearn.metrics import classification_report_imbalanced

In [34]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.62      0.65      0.02      0.64      0.40        87
   low_risk       1.00      0.65      0.62      0.79      0.64      0.41     17118

avg / total       0.99      0.65      0.62      0.79      0.64      0.41     17205



SMOTE

In [35]:
from imblearn.over_sampling import SMOTE

In [36]:
X_resampled, y_resampled = SMOTE(random_state=1,sampling_strategy='auto').fit_resample(X_train, y_train)

In [37]:
model2 = LogisticRegression(solver='lbfgs', random_state=1)

In [38]:
model2.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [39]:
y_pred2 = model2.predict(X_test)

In [40]:
balanced_accuracy_score(y_test, y_pred2)

0.6302712208564487

In [41]:
confusion_matrix(y_test, y_pred2)

array([[   54,    33],
       [ 6165, 10953]], dtype=int64)

In [42]:
print(classification_report_imbalanced(y_test, y_pred2))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.62      0.64      0.02      0.63      0.40        87
   low_risk       1.00      0.64      0.62      0.78      0.63      0.40     17118

avg / total       0.99      0.64      0.62      0.78      0.63      0.40     17205



Undersampling

In [43]:
from imblearn.under_sampling import RandomUnderSampler

In [44]:
rus = RandomUnderSampler(random_state=1)

In [45]:
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [46]:
Counter(y_resampled)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [47]:
from sklearn.linear_model import LogisticRegression

In [48]:
model = LogisticRegression(solver='lbfgs', random_state=1)

In [49]:
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [50]:
from sklearn.metrics import balanced_accuracy_score

In [51]:
balanced_accuracy_score(y_test, y_pred)

0.6366972052004142

In [52]:
from sklearn.metrics import confusion_matrix

In [53]:
y_pred = model.predict(X_test)

In [54]:
confusion_matrix(y_test, y_pred)

array([[   54,    33],
       [ 5945, 11173]], dtype=int64)

In [55]:
from imblearn.metrics import classification_report_imbalanced

In [56]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.62      0.65      0.02      0.64      0.40        87
   low_risk       1.00      0.65      0.62      0.79      0.64      0.41     17118

avg / total       0.99      0.65      0.62      0.79      0.64      0.41     17205



Combination

In [57]:
from imblearn.combine import SMOTEENN

In [58]:
smote_enn = SMOTEENN(random_state=0)

In [59]:
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

In [60]:
from sklearn.linear_model import LogisticRegression

In [61]:
model3 = LogisticRegression(solver='lbfgs', random_state=1)

In [62]:
model3.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [63]:
from sklearn.metrics import balanced_accuracy_score

In [64]:
balanced_accuracy_score(y_test, y_pred)

0.6366972052004142

In [65]:
from sklearn.metrics import confusion_matrix

In [66]:
y_pred = model.predict(X_test)

In [67]:
confusion_matrix(y_test, y_pred)

array([[   54,    33],
       [ 5945, 11173]], dtype=int64)

In [68]:
from imblearn.metrics import classification_report_imbalanced

In [69]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.62      0.65      0.02      0.64      0.40        87
   low_risk       1.00      0.65      0.62      0.79      0.64      0.41     17118

avg / total       0.99      0.65      0.62      0.79      0.64      0.41     17205

